In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict
from data_loader import *

import pandas as pd
import numpy as np
import warnings
import torch
import os

warnings.filterwarnings("ignore")

In [2]:
train_df, dev_df, test_df = load_augmented_data('cckg')

label_encoder = LabelEncoder()
label_encoder.fit(train_df['fallacy type'])
train_df['fallacy type'] = label_encoder.transform(train_df['fallacy type'])
dev_df['fallacy type'] = label_encoder.transform(dev_df['fallacy type'])
test_df['fallacy type'] = label_encoder.transform(test_df['fallacy type'])
print(label_encoder.classes_)

['ad hominem' 'ad populum' 'appeal to emotion' 'circular reasoning'
 'equivocation' 'fallacy of credibility' 'fallacy of extension'
 'fallacy of logic' 'fallacy of relevance' 'false causality'
 'false dilemma' 'faulty generalization' 'intentional' 'no fallacy']


In [3]:
model_name = "howey/electra-base-mnli"
data_dir = 'data/argumentation/'

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_), ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

if torch.cuda.is_available():
    model = model.cuda()

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at howey/electra-base-mnli and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([14, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([14]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
training_args = TrainingArguments(
    output_dir=f'./results/{model_name.split("/")[-1]}',
    do_eval=True,
    do_train=True,
    num_train_epochs=5,
    save_total_limit=3,
    load_best_model_at_end=True,
    learning_rate=8.5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="steps",
    logging_strategy="steps",
    evaluation_strategy="steps",
    logging_steps=50,
    eval_steps=50,
    save_steps=50,
)
def compute_metrics(pred):
            labels = pred.label_ids
            preds = pred.predictions.argmax(-1)
            precision, recall, f1, _ = precision_recall_fscore_support(
                labels, preds, average="weighted"
            )
            acc = accuracy_score(labels, preds)
            return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [6]:
dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(dev_df),
    'test': Dataset.from_pandas(test_df)
})

def process(batch):
    inputs = tokenizer(batch["argument"], truncation=True, padding="max_length")
    return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "labels": batch["fallacy type"],
        }
    
tokenized_dataset = dataset.map(
    process,
    batched=True, 
    remove_columns=dataset["train"].column_names)
print(tokenized_dataset['train'])

Map:   0%|          | 0/8707 [00:00<?, ? examples/s]

Map:   0%|          | 0/1890 [00:00<?, ? examples/s]

Map:   0%|          | 0/1880 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 8707
})


In [7]:
trainer = Trainer(
            model=model,
            args=training_args,
            compute_metrics=compute_metrics,
            train_dataset=tokenized_dataset["train"],
            eval_dataset=tokenized_dataset["validation"],
            tokenizer=tokenizer,
        )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: luca-mouchel (lia_epfl). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disablin

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,1.915100,1.240423,0.654497,0.600504,0.595460,0.654497
100,1.074000,0.773148,0.810053,0.776321,0.802716,0.810053
150,0.739400,0.657451,0.813757,0.770923,0.782189,0.813757


KeyboardInterrupt: 

In [ ]:
trainer.save_model(f'models/{model_name.split("/")[-1]}')

In [ ]:
L = '{\n    "topic": "Cannabis should be legal.",\n    "fallacy type": "appeal to emotion",\n    "appeal to emotion fallacy counter": "No, think about all the innocent children who could be exposed to drugs if cannabis was legal!"\n}'


'Cannabis should be legal.'

In [ ]:
trainer.predict(tokenized_dataset['test'])